In [62]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from os.path  import basename
import tqdm
import urllib.request 
from PIL import Image 
import os

In [63]:
goodreadsDF = pd.read_csv("../Datasets/Image-Generation Data/goodreads_data.csv")

In [64]:
goodreadsDF

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...
...,...,...,...,...,...,...,...,...
9995,9995,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,How far would you go? If human society was gen...,"['Dystopia', 'Science Fiction', 'Post Apocalyp...",3.44,276,https://www.goodreads.com/book/show/22085400-b...
9996,9996,Dynamo,Eleanor Gustafson,Jeth Cavanaugh is searching for a new life alo...,[],4.23,60,https://www.goodreads.com/book/show/20862902-d...
9997,9997,The Republic of Trees,Sam Taylor,This dark fable tells the story of four Englis...,"['Fiction', 'Horror', 'Dystopia', 'Coming Of A...",3.29,383,https://www.goodreads.com/book/show/891262.The...
9998,9998,"Waking Up (Healing Hearts, #1)",Renee Dyer,For Adriana Monroe life couldn’t get any bette...,"['New Adult', 'Romance', 'Contemporary Romance...",4.13,263,https://www.goodreads.com/book/show/19347252-w...


In [42]:
images = "../Datasets/Image-Generation Data/bookcovers/"
os.makedirs(images,exist_ok=True)

In [77]:
goodreadsDF["isImageAvailable"] = False
goodreadsDF["imageLocation"] = ""

for index,rows in tqdm.tqdm(goodreadsDF.iterrows(),total=len(goodreadsDF)):

    book = rows["Book"].replace("/","")
    author = rows["Author"]
    link = rows["URL"]



    if os.path.isfile(f"{images}{book}-{author}.jpg"):
        goodreadsDF.at[index,"isImageAvailable"] = True
        goodreadsDF.at[index,"imageLocation"] = f"{images}{book}-{author}.jpg"
        continue



    response = requests.get(link)
    soup = BeautifulSoup(response.content)
    
    imageLink = soup.find("div", { "class" : "BookCover__image" })
    
    try:
        imageLink = imageLink.select("img[src^=http]")[0]["src"]
        urllib.request.urlretrieve(imageLink, f"{images}{book}-{author}.jpg")

        goodreadsDF.at[index,"isImageAvailable"] = True
        goodreadsDF.at[index,"imageLocation"] = f"{images}{book}-{author}.jpg"
        
    except Exception as e:
        print(f"Book - {book}")
        print(f"Author - {author}")
        print(f"URL - {link}")
        print(f"Image Link - {imageLink}")

        goodreadsDF.at[index,"isImageAvailable"] = False
        goodreadsDF.at[index,"imageLocation"] = None

        print("Exception occured for {} : {}".format(book, repr(e)))

  2%|███▌                                                                                                                                                                  | 216/10000 [00:00<00:14, 689.17it/s]

Book - The Constitution of the United States of America
Author - Founding Fathers
URL - https://www.goodreads.com/book/show/89959.The_Constitution_of_the_United_States_of_America
Image Link - None
Exception occured for The Constitution of the United States of America : AttributeError("'NoneType' object has no attribute 'select'")


  6%|█████████▍                                                                                                                                                            | 566/10000 [00:02<00:43, 218.17it/s]

Book - The Celestine Prophecy (Celestine Prophecy, #1)
Author - James Redfield
URL - https://www.goodreads.com/book/show/13103.The_Celestine_Prophecy
Image Link - https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1166542346i/13103.jpg
Exception occured for The Celestine Prophecy (Celestine Prophecy, #1) : <HTTPError 403: 'Forbidden'>


 11%|██████████████████▉                                                                                                                                                  | 1146/10000 [00:03<00:22, 389.33it/s]

...

Book - The key to successful, healthy weight loss
Author - DietKart
URL - https://www.goodreads.com/book/show/18481121-the-key-to-successful-healthy-weight-loss
Image Link - None
Exception occured for The key to successful, healthy weight loss : AttributeError("'NoneType' object has no attribute 'select'")


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 9767/10000 [00:13<00:00, 515.83it/s]

Book - Lucy's Big Mess
Author - Kim Yannayon
URL - https://www.goodreads.com/book/show/18811555-lucy-s-big-mess
Image Link - None
Exception occured for Lucy's Big Mess : AttributeError("'NoneType' object has no attribute 'select'")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:13<00:00, 716.16it/s]

Book - Morning Glory
Author - LaVyrle Spencer
URL - https://www.goodreads.com/book/show/6510802-morning-glory
Image Link - None
Exception occured for Morning Glory : AttributeError("'NoneType' object has no attribute 'select'")


In [76]:
goodreadsDF.to_csv("../Datasets/Image-Generation Data/goodreads_data_with_images.csv",index=False)